In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("projjal1/human-conversation-training-data")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/human-conversation-training-data


In [ ]:
file_path = path + "/human_chat.txt"
with open(file_path, 'r') as f:
    file_content = f.readlines()
data=''
for line in file_content:
  data+=line[9:]


In [ ]:
import numpy as np

In [ ]:
data

'Hi!\nWhat is your favorite holiday?\none where I get to meet lots of different people.\nWhat was the most number of people you have ever met during a holiday?\nHard to keep a count. Maybe 25.\nWhich holiday was that?\nI think it was Australia\nDo you still talk to the people you met?\nNot really. The interactions are usually short-lived but it\'s fascinating to learn where people are coming from and what matters to them\nYea, me too. I feel like God often puts strangers in front of you, and gives you an opportunity to connect with them in that moment in deeply meaningful ways. Do you ever feel like you know things about strangers without them telling you?\nwhat do you mean?\nI think it\'s like a 6th sense, often seen as "cold readings" to people, but can be remarkably accurate. I once sat next to a man in a coffee and I felt a pain in my back. I asked the stranger if he had a pain. It turns out that he did in the exact spot, and said he pulled a muscle while dancing at a party. I had 

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()
tokenizer.fit_on_texts([data])

In [ ]:
word_index=tokenizer.word_index #words are indexed
total_words=len(word_index)+1
print(total_words)

2809


In [ ]:
input_sequences=[]  #input x
output=[]           #output y
for sentense in data.split('\n'):
  tokenised_sentense=tokenizer.texts_to_sequences([sentense])[0]
  for i in range(1,len(tokenised_sentense)):
    n_gram_sequence=tokenised_sentense[:i+1]

    input_sequences.append(n_gram_sequence[:-1])
    output.append(n_gram_sequence[-1])


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_len=max([len(x) for x in input_sequences])    #maximum is 150
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))

In [ ]:
from keras.utils import to_categorical
categorical_output=to_categorical(output,num_classes=total_words)
print(len(categorical_output))

18070


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.regularizers import l1_l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

model = Sequential()
# Input Layer
model.add(Embedding(
    input_dim=total_words,
    output_dim=64,  # Increased embedding dimension
    # input_length=max_sequence_len - 1,  # Corrected input length - Removed deprecated argument
    mask_zero=True              # Ignore padding indices
))

# LSTM Layer 1 wrapped in Bidirectional with regularization
model.add(Bidirectional(
    LSTM(
        units=128, # Reduced units
        return_sequences=False,
        dropout=0.3,
        recurrent_dropout=0.2,
        kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4),
        recurrent_regularizer=l1_l2(l1=1e-5, l2=1e-4)
    )
))

model.add(Dense(32,activation='relu'))

# Output Layer with regularization
model.add(Dense(
    units=total_words,       # Match vocabulary size
    activation='softmax',

))

model.compile(
    loss='categorical_crossentropy',  # Efficient for large vocab
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
model.build(input_shape=(None, max_sequence_len-1))
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 149, 64)        │       179,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ (None, 256)            │       197,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 2809)           │        92,697 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 478,329 (1.82 MB)

 Trainable params: 478,329 (1.82 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

model.fit(input_sequences,categorical_output, epochs=20, validation_split=0.2, batch_size=256)

Epoch 1/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 139s 2s/step - accuracy: 0.0295 - loss: 7.7918 - val_accuracy: 0.0277 - val_loss: 6.7790
Epoch 2/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 129s 2s/step - accuracy: 0.0281 - loss: 6.4887 - val_accuracy: 0.0277 - val_loss: 6.7373
Epoch 3/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.0308 - loss: 6.3135 - val_accuracy: 0.0310 - val_loss: 6.7335
Epoch 4/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.0395 - loss: 6.2381 - val_accuracy: 0.0296 - val_loss: 6.6931
Epoch 5/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.0382 - loss: 6.1654 - val_accuracy: 0.0423 - val_loss: 6.6590
Epoch 6/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.0381 - loss: 6.0880 - val_accuracy: 0.0418 - val_loss: 6.7123
Epoch 7/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.0407 - loss: 6.0351 - val_accuracy: 0.0515 - val_loss: 6.7115
Epoch 8/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.0474 - loss: 5.9389 - val_accuracy: 0.0529 - v

In [ ]:
model.fit(input_sequences,categorical_output, epochs=20, validation_split=0.2, batch_size=256)

Epoch 21/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.1179 - loss: 5.1076 - val_accuracy: 0.1060 - val_loss: 7.2250
Epoch 22/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.1269 - loss: 5.0352 - val_accuracy: 0.1093 - val_loss: 7.2387
Epoch 23/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.1257 - loss: 4.9890 - val_accuracy: 0.1151 - val_loss: 7.3030
Epoch 24/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.1295 - loss: 4.9410 - val_accuracy: 0.1148 - val_loss: 7.3155
Epoch 25/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.1341 - loss: 4.8875 - val_accuracy: 0.1168 - val_loss: 7.4150
Epoch 26/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.1346 - loss: 4.8303 - val_accuracy: 0.1140 - val_loss: 7.5711
Epoch 27/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.1395 - loss: 4.7759 - val_accuracy: 0.1204 - val_loss: 7.6143
Epoch 28/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.1470 - loss: 4.7216 - val_accuracy: 0.

In [ ]:
test_sentense=input()
for i in range(10):
  tokenised_sentense=tokenizer.texts_to_sequences([test_sentense])[0]
  input_test_sequence=np.array(pad_sequences([tokenised_sentense],maxlen=max_sequence_len-1,padding='pre'))
  predicted_word=model.predict(input_test_sequence)
  predicted_word=np.argmax(predicted_word)
  for word,index in tokenizer.word_index.items():
    if index==predicted_word:
      test_sentense+=" "+word
      print(word)
      break

print(test_sentense)


hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
am
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
a
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
lot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
of
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
work
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
but
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
think
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
to
hi i am a lot of work but i think to


In [ ]:
import matplotlib.pyplot as plt
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

TypeError: 'History' object is not subscriptable

In [ ]:
!pip install tensorflow